### *This file allows to reproduce FigS4D-F*

# **Useful packages and functions**

In [ ]:
using DifferentialEquations, Plots, Plots.PlotMeasures, LaTeXStrings, Random, Dierckx, DelimitedFiles
using Interpolations
include("STG_kinetics.jl") # Loading of STG kinetics of gating variables
include("STG_models.jl") # Loading of STG model
include("STG_utils.jl") # Loading of some utils functions
include("STG_gs_derivatives.jl") # Loading of X_inf derivatives
include("STG_DIC.jl") # Loading of the DIC and compensation algorithm
include("STG_neuromodulation.jl") # Loading of the neuromodulation cells functions
include("STG_large_simulations.jl"); # Loading of the neuromodulation cells functions

# **Global variables**

In [ ]:
# Definition of simulation time (in ms)
const Tfinal = 300000
const tspan  = (0.0, Tfinal)

# Definition of reversal potential values (in mV) and membrane capacitance
const VNa   = 50. # Sodium reversal potential
const VK    = -80. # Potassium reversal potential
const VCa   = 80. # Calcium reversal potential
const VH    = -20. # Reversal potential for the H-current (permeable to both sodium and potassium ions)
const Vleak = -50. # Reversal potential of leak channels
const C     = 1. # Membrane capacitance

# Definition of voltage range for the DICs
const Vmin = -60 
const Vmax = 0
const V    = range(Vmin, stop=Vmax, step=0.01)

# Definition of the number of cells in the random set
const ncells = 200

# Modifying backend GR attributes
gr(guidefontsize=18, legendfontsize=12, margin=5Plots.mm, grid=false)
myApple = RGBA(187/255, 206/255, 131/255, 1)
mySalmon = RGBA(243/255, 124/255, 130/255)
myYellow = RGBA(228/255, 205/255, 121/255, 1)
myBlue = RGBA(131/255, 174/255, 218/255, 1)
myDarkBlue = RGBA(114/255, 119/255, 217/255, 1)
myOrange = RGBA(241/255, 175/255, 113/255, 1)
myPink = RGBA(243/255, 124/255, 130/255, 1)
myPurple = RGBA(169/255, 90/255, 179/255, 1)
myGreen = RGBA(132/255, 195/255, 168/255, 1)
myRed = RGBA(158/255, 3/255, 8/255, 1)
myGray = RGBA(150/255, 150/255, 150/255, 1)
myLightBlue = RGBA(127/255, 154/255, 209/255, 1)
default(fmt = :png)

# Moving average function
moving_average(vs, n, padding) = [sum(vs[i:(i+n-1)])/n for i in 1:padding:(length(vs)-(n-1))];

# **Time simulation of FigS4D-E (uncomment to run, might take a while)**

In [ ]:
# Fixing random seed
Random.seed!(544)

# Initial firing pattern
guth = 4.
Vth = -50.
(g_all_init, ICs_th_init) = degeneracy_fixDICs_neuromod(ncells, 5., guth, Vth);
# create a spiking set with max variability in gCaS and gA

In [ ]:
# Definition of parameters
α = 5e-3 # Rate of transfer between intracellular and membrane
β = 5e-3 # Rate of degradation of intracellular proteins
Kp = 3e-4 # Proportional gain
Ki = 5e-6 # Integral gain
Kt = β / Ki # Anti-windup gain
gsth_sim(t) = 5 - 13 * pulse(t, Tfinal/2, Tfinal) # Reference
guth_sim(t) = 4.
u_maxCaS = 1e7 # Maximum value of actuator
u_maxH = 1e7

# Definition of homeostatic parameters
tau_g = 100 # Conductance time constant
tau_Na = 600 # Sodium integral action time constants
Ca_tgt(t) = 125. # Calcium target

# Input current definition
Iapp = 0.

tt = 0. : 0.2 : Tfinal

window_size_s = 5
padding_s = 0.2
window_size_ = Int64(round(window_size_s*1000*length(tt)/(Tfinal)))
padding_ = Int64(round(padding_s*1000*length(tt)/(Tfinal)))
tt_moving_average_ = window_size_s/2 : padding_s : Tfinal/1000 - window_size_s/2
tt_moving_average_plot = range(0, Tfinal/1e3, length(tt_moving_average_));

In [ ]:
# gNa_matrix, gCaT_matrix, gCaS_matrix, gA_matrix, gKCa_matrix, gKd_matrix, gH_matrix, 
# gleak_matrix, Ca_ma_matrix = simulate_STG_population_H(g_all_init, Iapp, tau_Na, tau_g, Ca_tgt,
#                                                      C, α, β, Kp, Ki, Kt, gsth_sim, guth_sim,
#                                                      ICs_th_init, u_maxCaS, u_maxH, tt)

In [ ]:
# writedlm("./data/gNa_matrix_H.dat", gNa_matrix)
# writedlm("./data/gCaT_matrix_H.dat", gCaT_matrix)
# writedlm("./data/gCaS_matrix_H.dat", gCaS_matrix)
# writedlm("./data/gA_matrix_H.dat", gA_matrix)
# writedlm("./data/gKCa_matrix_H.dat", gKCa_matrix)
# writedlm("./data/gKd_matrix_H.dat", gKd_matrix)
# writedlm("./data/gH_matrix_H.dat", gH_matrix)
# writedlm("./data/gleak_matrix_H.dat", gleak_matrix)
# writedlm("./data/Ca_ma_matrix_H.dat", Ca_ma_matrix)

# **Plotting FigS4D-E**

In [ ]:
gNa_matrix = readdlm("./data/gNa_matrix_H.dat")
gCaT_matrix = readdlm("./data/gCaT_matrix_H.dat")
gCaS_matrix = readdlm("./data/gCaS_matrix_H.dat")
gA_matrix = readdlm("./data/gA_matrix_H.dat")
gKCa_matrix = readdlm("./data/gKCa_matrix_H.dat")
gKd_matrix = readdlm("./data/gKd_matrix_H.dat")
gH_matrix = readdlm("./data/gH_matrix_H.dat")
gleak_matrix = readdlm("./data/gleak_matrix_H.dat")
Ca_ma_matrix = readdlm("./data/Ca_ma_matrix_H.dat");

In [ ]:
p1 = plot(ylims=(1e-1, 1e5), yticks=([1e-1, 1e1, 1e3, 1e5], [L"10^{-1}", L"10^{1}", L"10^{3}", L"10^{5}"]), 
          guidefontsize=18, xticks=([0, 100, 200, 300], [L"0", L"100", L"200", L"300"]), tickfontsize=15, 
          size=(600, 300), xlims=(0, 300))

xlabel!(L"t\,\mathrm{(s)}")
ylabel!(L"\bar{g}\,\mathrm{(mS/cm^2)}")

for i = 1 : ncells
    plot!(tt[2:5000:end]./1e3, gNa_matrix[i, :], color=myApple, linewidth=1.5,
          legend=false, alpha=0.1, yaxis=:log)
    plot!(tt[2:5000:end]./1e3, gCaT_matrix[i, :], color=myYellow, linewidth=1.5,
          legend=false, alpha=0.1, yaxis=:log)
    plot!(tt[2:5000:end]./1e3, gA_matrix[i, :], color=myPurple, linewidth=1.5,
          legend=false, alpha=0.2, yaxis=:log)
    plot!(tt[2:5000:end]./1e3, gCaS_matrix[i, :], color=myGreen, linewidth=1.5,
          legend=false, alpha=0.1, yaxis=:log)
    plot!(tt[2:5000:end]./1e3, gKCa_matrix[i, :], color=myBlue, linewidth=1.5,
          legend=false, alpha=0.1, yaxis=:log)
    plot!(tt[2:5000:end]./1e3, gKd_matrix[i, :], color=myPink, linewidth=1.5,
          legend=false, alpha=0.1, yaxis=:log)
    plot!(tt[2:5000:end]./1e3, gH_matrix[i, :], color=myOrange, linewidth=1.5,
          legend=false, alpha=0.1, yaxis=:log)
    plot!(tt[2:5000:end]./1e3, gleak_matrix[i, :], color=myDarkBlue, linewidth=1.5,
          legend=false, alpha=0.1, yaxis=:log)
end

display(p1)
# savefig(p1, "./figures/g_all_H.pdf")

In [ ]:
tt_moving_average_plot = range(0, 300, length=length(Ca_ma_matrix[1, :]))
p1b = plot(size=(600, 300), ylims=(0, 600), yticks=([0, 200, 400, 600], [L"0", L"200", L"400", L"600"]), 
           xlims=(0, 300), guidefontsize=18, 
           xticks=([0, 100, 200, 300], [L"0", L"100", L"200", L"300"]), tickfontsize=15)

ylabel!(L"\overline{[Ca]}")
xlabel!(L"t\,\mathrm{(s)}")

for i = 1 : ncells
    plot!(tt_moving_average_plot, Ca_ma_matrix[i, :], linewidth=1.5, color=:black, 
          alpha=0.05, legend=false)
end

plot!([0, 300], [125, 125], color=:firebrick1, linestyle=:dashdot, linewidth=1.5)

display(p1b)
# savefig(p1b, "./figures/Ca_ma_H.pdf")

# **Violin plots computations FigS4F**

In [ ]:
tt_g = tt[2:5000:end]./1e3
i_acute = findall(tt_g .> 150 .&& tt_g .< 151)[1]
frequency_before = zeros(ncells)
for i = 1 : ncells
    gNa = gNa_matrix[i, i_acute-1]
    gCaT = gCaT_matrix[i, i_acute-1]
    gCaS = gCaS_matrix[i, i_acute-1]
    gA = gA_matrix[i, i_acute-1]
    gKCa = gKCa_matrix[i, i_acute-1]
    gKd = gKd_matrix[i, i_acute-1]
    gH = gH_matrix[i, i_acute-1]
    gleak = gleak_matrix[i, i_acute-1]
    
    p = (Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C)
    # Initial conditions
    V0  = -70.
    Ca0 = 0.5
    x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), hCaS_inf(V0),
            mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
    # Simulation
    prob = ODEProblem(STG_ODE, x0, (0, Tfinal/20), p) # Describing the problem
    sol = solve(prob) # Solving the problem

    tt_short = Tfinal/100 : 0.1 : Tfinal/20;
    x = sol(tt_short)
    V_short = x[1, :]
    frequency_before[i] = extract_frequency(V_short, tt_short)
#     display(plot(tt_short, V_short))
end
violin(frequency_before)
ylims!((0, 20))

In [ ]:
tt_g = tt[2:5000:end]./1e3
i_acute = findall(tt_g .> 180 .&& tt_g .< 181)[1]
burstiness_acute = zeros(ncells)
for i = 1 : ncells
    gNa = gNa_matrix[i, i_acute]
    gCaT = gCaT_matrix[i, i_acute]
    gCaS = gCaS_matrix[i, i_acute]
    gA = gA_matrix[i, i_acute]
    gKCa = gKCa_matrix[i, i_acute]
    gKd = gKd_matrix[i, i_acute]
    gH = gH_matrix[i, i_acute]
    gleak = gleak_matrix[i, i_acute]
    
    p = (Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C)
    # Initial conditions
    V0  = -70.
    Ca0 = 0.5
    x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), hCaS_inf(V0),
            mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
    # Simulation
    prob = ODEProblem(STG_ODE, x0, (0, Tfinal/20), p) # Describing the problem
    sol = solve(prob) # Solving the problem

    tt_short = Tfinal/100 : 0.1 : Tfinal/20;
    x = sol(tt_short)
    V_short = x[1, :]
    burstiness_acute[i], _, _, _ = extract_burstiness(V_short, tt_short)
#     display(plot(tt_short, V_short))
end
violin(burstiness_acute)
ylims!((0, 30000))

In [ ]:
burstiness_end = zeros(ncells)
for i = 1 : ncells
    gNa = gNa_matrix[i, end]
    gCaT = gCaT_matrix[i, end]
    gCaS = gCaS_matrix[i, end]
    gA = gA_matrix[i, end]
    gKCa = gKCa_matrix[i, end]
    gKd = gKd_matrix[i, end]
    gH = gH_matrix[i, end]
    gleak = gleak_matrix[i, end]
    
    p = (Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C)
    # Initial conditions
    V0  = -70.
    Ca0 = 0.5
    x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), hCaS_inf(V0),
            mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
    # Simulation
    prob = ODEProblem(STG_ODE, x0, (0, Tfinal/20), p) # Describing the problem
    sol = solve(prob) # Solving the problem

    tt_short = Tfinal/100 : 0.1 : Tfinal/20;
    x = sol(tt_short)
    V_short = x[1, :]
    burstiness_end[i], _, _, _ = extract_burstiness(V_short, tt_short)
    if isnan(burstiness_end[i])
        burstiness_end[i] = 0
    end
#     display(plot(tt_short, V_short))
end

violin(burstiness_end)
ylims!((0, 30000))

In [ ]:
# writedlm("./data/frequency_before_H.dat", frequency_before)
# writedlm("./data/burstiness_acute_H.dat", burstiness_acute)
# writedlm("./data/burstiness_end_H.dat", burstiness_end)

# **Violin plots FigS4F**

In [ ]:
frequency_before = readdlm("./data/frequency_before_H.dat")
burstiness_acute = readdlm("./data/burstiness_acute_H.dat")
burstiness_end = readdlm("./data/burstiness_end_H.dat");

In [ ]:
violin_f = violin([frequency_before], label="", color=myGray, grid=false,
             yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm,
             markerstrokewidth=0., yticks=([0, 10, 20], [L"0", L"10", L"20"]), tickfontsize=15, 
             xticks=([1,], [L"t_\mathrm{before}",]), size=(300, 300))

ylabel!(L"\mathrm{frequency}\,\mathrm{(Hz)}")
ylims!((0, 20))
display(violin_f)
# savefig(violin_f, "./figures/violins_frequency_H.pdf")

In [ ]:
violin_after = violin(burstiness_acute, label="", color=myGray, grid=false,
             yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm,
             markerstrokewidth=0., yticks=([0, 10000, 20000], [L"0", L"10000", L"20000"]), tickfontsize=15, 
             xticks=([1], [L"t_\mathrm{after}"]), size=(300, 300))

ylabel!(L"\mathrm{burstiness}\,\mathrm{(Hz^2)}")
ylims!((0, 20000))
display(violin_after)
# savefig(violin_after, "./figures/violins_burstiness_after_H.pdf")

In [ ]:
violin_after = violin(burstiness_end, label="", color=myGray, grid=false,
             yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm,
             markerstrokewidth=0., yticks=([0, 10000, 20000], [L"0", L"10000", L"20000"]), tickfontsize=15, 
             xticks=([1], [L"t_\mathrm{compensated}"]), size=(300, 300))

ylabel!(L"\mathrm{burstiness}\,\mathrm{(Hz^2)}")
ylims!((0, 20000))
display(violin_after)
# savefig(violin_after, "./figures/violins_burstiness_compensated_H.pdf")

# **Voltage traces at t compensated FigS4F**

In [ ]:
tt_g = tt[2:5000:end]./1e3
i_acute = findall(tt_g .> 150 .&& tt_g .< 151)[1]
j = 54
gNa = gNa_matrix[j, end]
gCaT = gCaT_matrix[j, end]
gCaS = gCaS_matrix[j, end]
gA = gA_matrix[j, end]
gKCa = gKCa_matrix[j, end]
gKd = gKd_matrix[j, end]
gH = gH_matrix[j, end]
gleak = gleak_matrix[j, end]

p = (Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C)
# Initial conditions
V0  = -70.
Ca0 = 0.5
x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), hCaS_inf(V0),
        mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
# Simulation
prob = ODEProblem(STG_ODE, x0, (0, Tfinal/20), p) # Describing the problem
sol = solve(prob) # Solving the problem

tt_short = Tfinal/100 : 0.1 : Tfinal/20;
x = sol(tt_short)
V_short = x[1, :]

p = plot(tt_short/1e3, V_short, xlims=(5.63, 6.20), xticks=false, 
         yticks=false, axis=false, linewidth=2.5, legend=false, size=(600, 200), 
         color=myGray, margins=0px)
display(p)
# savefig(p, "./figures/STG_end1_H.pdf")

In [ ]:
tt_g = tt[2:5000:end]./1e3
i_acute = findall(tt_g .> 150 .&& tt_g .< 151)[1]
j = 102
gNa = gNa_matrix[j, end]
gCaT = gCaT_matrix[j, end]
gCaS = gCaS_matrix[j, end]
gA = gA_matrix[j, end]
gKCa = gKCa_matrix[j, end]
gKd = gKd_matrix[j, end]
gH = gH_matrix[j, end]
gleak = gleak_matrix[j, end]

p = (Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C)
# Initial conditions
V0  = -70.
Ca0 = 0.5
x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), hCaS_inf(V0),
        mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
# Simulation
prob = ODEProblem(STG_ODE, x0, (0, Tfinal/20), p) # Describing the problem
sol = solve(prob) # Solving the problem

tt_short = Tfinal/100 : 0.1 : Tfinal/20;
x = sol(tt_short)
V_short = x[1, :]

p = plot(tt_short/1e3, V_short, xlims=(5.6, 6.15), xticks=false, 
         yticks=false, axis=false, linewidth=2.5, legend=false, size=(600, 200), 
         color=myGray, margins=0px)
display(p)
# savefig(p, "./figures/STG_end2_H.pdf")

In [ ]:
tt_g = tt[2:5000:end]./1e3
i_acute = findall(tt_g .> 150 .&& tt_g .< 151)[1]
j = 181
gNa = gNa_matrix[j, end]
gCaT = gCaT_matrix[j, end]
gCaS = gCaS_matrix[j, end]
gA = gA_matrix[j, end]
gKCa = gKCa_matrix[j, end]
gKd = gKd_matrix[j, end]
gH = gH_matrix[j, end]
gleak = gleak_matrix[j, end]

p = (Iapp, gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak, C)
# Initial conditions
V0  = -70.
Ca0 = 0.5
x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), hCaS_inf(V0),
        mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]
# Simulation
prob = ODEProblem(STG_ODE, x0, (0, Tfinal/20), p) # Describing the problem
sol = solve(prob) # Solving the problem

tt_short = Tfinal/100 : 0.1 : Tfinal/20;
x = sol(tt_short)
V_short = x[1, :]

p = plot(tt_short/1e3, V_short, xlims=(5.62, 6.20), xticks=false, 
         yticks=false, axis=false, linewidth=2.5, legend=false, size=(600, 200), 
         color=myGray, margins=0px)
display(p)
# savefig(p, "./figures/STG_end3_H.pdf")